In [1]:
import pandas as pd
from pathlib import Path
import numpy as np
from sklearn.cluster import KMeans
import haversine as hs
from haversine import Unit
import numpy as np
import pickle


/tmp/ipykernel_576759/1925167382.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
input_data_folder_base = Path("../results/")
input_data_folder = Path.cwd() / input_data_folder_base
overall_collection_file_name = "overall_collection_2023.p"
key_parameter_file_name = "key_parameter_2023.p"
cluster_positions_file_name = "cluster_positions_2023.p"
print(f"Reading raw pandas files from \n    {input_data_folder}")

output_data_folder = input_data_folder
print(f"Writing merged pandas file to \n    {output_data_folder}")

Reading raw pandas files from 
    /home/sebastian/Nextcloud/Projekte/2021_DAAD_Brasilien/2023/RoBiMo_Trop_DataSet/notebooks/../results
Writing merged pandas file to 
    /home/sebastian/Nextcloud/Projekte/2021_DAAD_Brasilien/2023/RoBiMo_Trop_DataSet/notebooks/../results


In [3]:
df_results = pd.read_pickle(input_data_folder / overall_collection_file_name)
df_results.head(3)

,lat_est,lon_est,yaw_est,hdop,nsats,lat_meas,lon_meas,depth,experiment_location,meas_running,corresponding_meas_file,CO2(ppm),airtemp in(degreeC),humidity in(rH),pressure in(mbar),airtemp out(degreeC),humidity out(rH),pressure out(mbar),PAR(umol m-2s-1),H2O temp(degreeC)
datetime_AMT,,,,,,,,,,,,,,,,,,,,
2023-03-04 20:00:00,-3.290738,-60.164893,111.314,2.59,6.0,-3.290751,-60.164883,7.617662,Iranduba,False,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-03-04 20:00:01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Iranduba,False,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-03-04 20:00:02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Iranduba,False,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
df_results.dropna(subset = ['lat_est', 'lon_est'], inplace=True)
df_results

,lat_est,lon_est,yaw_est,hdop,nsats,lat_meas,lon_meas,depth,experiment_location,meas_running,corresponding_meas_file,CO2(ppm),airtemp in(degreeC),humidity in(rH),pressure in(mbar),airtemp out(degreeC),humidity out(rH),pressure out(mbar),PAR(umol m-2s-1),H2O temp(degreeC)
datetime_AMT,,,,,,,,,,,,,,,,,,,,
2023-03-04 20:00:00,-3.290738,-60.164893,111.314,2.59,6.0,-3.290751,-60.164883,7.617662,Iranduba,False,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-03-11 18:30:37,-3.290886,-60.165142,162.084,0.83,15.0,-3.290886,-60.165141,5.512197,Iranduba,False,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-03-11 18:30:38,-3.290886,-60.165140,172.000,0.78,17.0,-3.290886,-60.165140,5.475826,Iranduba,False,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-03-11 18:30:39,-3.290885,-60.165141,166.150,0.78,17.0,-3.290886,-60.165141,5.475826,Iranduba,False,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-03-11 18:30:40,-3.290882,-60.165145,161.714,0.78,17.0,-3.290883,-60.165144,5.473805,Iranduba,False,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-03-07 15:47:39,-3.253049,-60.219045,11.708,0.56,24.0,-3.253050,-60.219045,6.469961,Caldeirao,False,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-03-07 15:47:40,-3.253043,-60.219046,4.492,0.56,24.0,-3.253044,-60.219046,6.476023,Caldeirao,False,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-03-07 15:47:41,-3.253038,-60.219049,4.738,0.55,25.0,-3.253039,-60.219048,6.435611,Caldeirao,False,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
cluster_count = {
    "Balbina": 3,
    "Jandira":3,
    "Caldeirao": 6,
    "Iranduba": 4,
}

cluster_position = {
    "Balbina": np.nan,
    "Jandira": np.nan,
    "Caldeirao": np.nan,
    "Iranduba": np.nan,
}

def run_k_means(location_name, df_sep_pos, n_clusters):
    positions = df_sep_pos[['lat_est','lon_est']]
    kmeans = KMeans(n_clusters=n_clusters, n_init=10).fit(positions)
    centroids = kmeans.cluster_centers_
    return centroids

def dist(lat_0, lon_0, lat_1, lon_1):
    return hs.haversine((lat_0, lon_0), (lat_1, lon_1), unit=Unit.METERS)

df_results["position"] = np.nan
for entry in cluster_count:
    print(f"{entry} ...")
    print(f"   ... running cluster analysis")
    location_name = entry
    df_sep_pos = df_results[(df_results.experiment_location == location_name) &
                            (df_results.meas_running)]
    centroids = run_k_means(location_name, df_sep_pos,
                            cluster_count[location_name])
    print(centroids)
    cluster_position[entry] = centroids

    print(f"   ... calculating distances")
    for id_row, row in df_results.iterrows():
        if ((row.experiment_location == location_name) & (row.meas_running)):
            smallest_distance = np.inf
            id_of_smallest_dist = 100000
            for id_pos, position in enumerate(centroids):
                distance = dist(row['lat_est'], row['lon_est'], position[0], position[1])
                if distance < smallest_distance:
                    id_of_smallest_dist = id_pos
                    smallest_distance = distance
            df_results.at[id_row, "position"] = id_of_smallest_dist

Balbina ...
   ... running cluster analysis


[[ -1.90938492 -59.46105017]
 [ -1.90498015 -59.46979682]
 [ -1.90367786 -59.46055068]]
   ... calculating distances


Jandira ...
   ... running cluster analysis


[[ -3.23674311 -60.24114295]
 [ -3.23378505 -60.24179794]
 [ -3.2354515  -60.2408596 ]]
   ... calculating distances


Caldeirao ...
   ... running cluster analysis


[[ -3.24645146 -60.21718762]
 [ -3.25525976 -60.21942943]
 [ -3.25665423 -60.21853891]
 [ -3.25408879 -60.21902   ]
 [ -3.2460189  -60.21627407]
 [ -3.25595593 -60.21851002]]
   ... calculating distances


Iranduba ...
   ... running cluster analysis


[[ -3.29182389 -60.1638196 ]
 [ -3.29371632 -60.16349051]
 [ -3.29160827 -60.16397365]
 [ -3.29146567 -60.16393856]]
   ... calculating distances


In [6]:
df_results.loc[df_results["corresponding_meas_file"] == "040323-151128-ADC.p", "position"] = 3.0
aux = np.append(cluster_position['Jandira'],
          [[df_results[df_results["corresponding_meas_file"] == "040323-151128-ADC.p"].lat_est[0],
           df_results[df_results["corresponding_meas_file"] == "040323-151128-ADC.p"].lon_est[0]]],
           axis=0)

cluster_position['Jandira'] = aux
cluster_position
           

/tmp/ipykernel_576759/1762901757.py:3: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  [[df_results[df_results["corresponding_meas_file"] == "040323-151128-ADC.p"].lat_est[0],
/tmp/ipykernel_576759/1762901757.py:4: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  df_results[df_results["corresponding_meas_file"] == "040323-151128-ADC.p"].lon_est[0]]],


{'Balbina': array([[ -1.90938492, -59.46105017],
        [ -1.90498015, -59.46979682],
        [ -1.90367786, -59.46055068]]),
 'Jandira': array([[ -3.23674311, -60.24114295],
        [ -3.23378505, -60.24179794],
        [ -3.2354515 , -60.2408596 ],
        [ -3.2338013 , -60.24178698]]),
 'Caldeirao': array([[ -3.24645146, -60.21718762],
        [ -3.25525976, -60.21942943],
        [ -3.25665423, -60.21853891],
        [ -3.25408879, -60.21902   ],
        [ -3.2460189 , -60.21627407],
        [ -3.25595593, -60.21851002]]),
 'Iranduba': array([[ -3.29182389, -60.1638196 ],
        [ -3.29371632, -60.16349051],
        [ -3.29160827, -60.16397365],
        [ -3.29146567, -60.16393856]])}

In [7]:
result_file = output_data_folder / Path(overall_collection_file_name)
print(f"... saving common results to {  Path(overall_collection_file_name)}")
df_results.to_pickle(result_file)

... saving common results to overall_collection_2023.p


In [8]:
result_file = output_data_folder / Path(cluster_positions_file_name)
with open(result_file, 'wb') as f:
    pickle.dump(cluster_position, f)

In [9]:
df_results.shape

(201618, 21)